In [1]:
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import random
import itertools
import math
import os

In [2]:
import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## NYC Taxi Data

#### Download the Trip Record Data

In [3]:
'''
for month in range(1,2):
    urllib.request.urlretrieve("https://s3.amazonaws.com/nyc-tlc/trip+data/"+ \
                               "yellow_tripdata_2018-{0:0=2d}.csv".format(month), 
                               "nyc.2018-{0:0=2d}.csv".format(month))
'''

'\nfor month in range(1,2):\n    urllib.request.urlretrieve("https://s3.amazonaws.com/nyc-tlc/trip+data/"+                                "yellow_tripdata_2018-{0:0=2d}.csv".format(month), \n                               "nyc.2018-{0:0=2d}.csv".format(month))\n'

In [4]:
nyc_df = pd.read_csv("datasets/nyc.2018-01.csv", parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'], nrows = 100)
nyc_df = nyc_df.set_index('tpep_pickup_datetime')
nyc_df.head()


VendorID tpep_dropoff_datetime  passenger_count  \
tpep_pickup_datetime                                                    
2018-01-01 00:21:05          1   2018-01-01 00:24:23                1   
2018-01-01 00:44:55          1   2018-01-01 01:03:05                1   
2018-01-01 00:08:26          1   2018-01-01 00:14:21                2   
2018-01-01 00:20:22          1   2018-01-01 00:52:51                1   
2018-01-01 00:09:18          1   2018-01-01 00:27:06                2   

                      trip_distance  RatecodeID store_and_fwd_flag  \
tpep_pickup_datetime                                                 
2018-01-01 00:21:05             0.5           1                  N   
2018-01-01 00:44:55             2.7           1                  N   
2018-01-01 00:08:26             0.8           1                  N   
2018-01-01 00:20:22            10.2           1                  N   
2018-01-01 00:09:18             2.5           1                  N   

                      PULocationID  DOLocationID  payment_type  fare_amount  \
tpep_pickup_datetime                                                          
2018-01-01 00:21:05             41            24             2          4.5   
2018-01-01 00:44:55            239           140             2         14.0   
2018-01-01 00:08:26            262           141             1          6.0   
2018-01-01 00:20:22            140           257             2         33.5   
2018-01-01 00:09:18            246           239             1         12.5   

                      extra  mta_tax  tip_amount  tolls_amount  \
tpep_pickup_datetime                                             
2018-01-01 00:21:05     0.5      0.5        0.00           0.0   
2018-01-01 00:44:55     0.5      0.5        0.00           0.0   
2018-01-01 00:08:26     0.5      0.5        1.00           0.0   
2018-01-01 00:20:22     0.5      0.5        0.00           0.0   
2018-01-01 00:09:18     0.5      0.5        2.75           0.0   

                      improvement_surcharge  total_amount  
tpep_pickup_datetime                                       
2018-01-01 00:21:05                     0.3          5.80  
2018-01-01 00:44:55                     0.3         15.30  
2018-01-01 00:08:26                     0.3          8.30  
2018-01-01 00:20:22                     0.3         34.80  
2018-01-01 00:09:18                     0.3         16.55

#### Clean Data

In [5]:
nyc_df = nyc_df.loc['2018-01']
nyc_df.head()


VendorID tpep_dropoff_datetime  passenger_count  \
tpep_pickup_datetime                                                    
2018-01-01 00:21:05          1   2018-01-01 00:24:23                1   
2018-01-01 00:44:55          1   2018-01-01 01:03:05                1   
2018-01-01 00:08:26          1   2018-01-01 00:14:21                2   
2018-01-01 00:20:22          1   2018-01-01 00:52:51                1   
2018-01-01 00:09:18          1   2018-01-01 00:27:06                2   

                      trip_distance  RatecodeID store_and_fwd_flag  \
tpep_pickup_datetime                                                 
2018-01-01 00:21:05             0.5           1                  N   
2018-01-01 00:44:55             2.7           1                  N   
2018-01-01 00:08:26             0.8           1                  N   
2018-01-01 00:20:22            10.2           1                  N   
2018-01-01 00:09:18             2.5           1                  N   

                      PULocationID  DOLocationID  payment_type  fare_amount  \
tpep_pickup_datetime                                                          
2018-01-01 00:21:05             41            24             2          4.5   
2018-01-01 00:44:55            239           140             2         14.0   
2018-01-01 00:08:26            262           141             1          6.0   
2018-01-01 00:20:22            140           257             2         33.5   
2018-01-01 00:09:18            246           239             1         12.5   

                      extra  mta_tax  tip_amount  tolls_amount  \
tpep_pickup_datetime                                             
2018-01-01 00:21:05     0.5      0.5        0.00           0.0   
2018-01-01 00:44:55     0.5      0.5        0.00           0.0   
2018-01-01 00:08:26     0.5      0.5        1.00           0.0   
2018-01-01 00:20:22     0.5      0.5        0.00           0.0   
2018-01-01 00:09:18     0.5      0.5        2.75           0.0   

                      improvement_surcharge  total_amount  
tpep_pickup_datetime                                       
2018-01-01 00:21:05                     0.3          5.80  
2018-01-01 00:44:55                     0.3         15.30  
2018-01-01 00:08:26                     0.3          8.30  
2018-01-01 00:20:22                     0.3         34.80  
2018-01-01 00:09:18                     0.3         16.55

## NYC Taxi Zones

In [6]:
'''
# Download the location Data
urllib.request.urlretrieve("https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip", "taxi_zones.zip")
with zipfile.ZipFile("taxi_zones.zip","r") as zip_ref:
    zip_ref.extractall("./datasets/taxi_zones/shape")
'''

'\n# Download the location Data\nurllib.request.urlretrieve("https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip", "taxi_zones.zip")\nwith zipfile.ZipFile("taxi_zones.zip","r") as zip_ref:\n    zip_ref.extractall("./datasets/taxi_zones/shape")\n'

In [7]:
def get_lat_lon(sf):
    content = []
    for sr in sf.shapeRecords():
        shape = sr.shape
        rec = sr.record
        loc_id = rec[shp_dic['LocationID']]
        
        x = (shape.bbox[0]+shape.bbox[2])/2
        y = (shape.bbox[1]+shape.bbox[3])/2
        
        content.append((loc_id, x, y))
    return pd.DataFrame(content, columns=["LocationID", "longitude", "latitude"])

#### Convert shape file to data frame

In [8]:
sf = shapefile.Reader("datasets/taxi_zones/shape/taxi_zones.shp")
fields_name = [field[0] for field in sf.fields[1:]]
shp_dic = dict(zip(fields_name, list(range(len(fields_name)))))
attributes = sf.records()
shp_attr = [dict(zip(fields_name, attr)) for attr in attributes]

loc_df = pd.DataFrame(shp_attr).join(get_lat_lon(sf).set_index("LocationID"), on="LocationID")
loc_df.head()

OBJECTID  Shape_Leng  Shape_Area                     zone  LocationID  \
0         1    0.116357    0.000782           Newark Airport           1   
1         2    0.433470    0.004866              Jamaica Bay           2   
2         3    0.084341    0.000314  Allerton/Pelham Gardens           3   
3         4    0.043567    0.000112            Alphabet City           4   
4         5    0.092146    0.000498            Arden Heights           5   

         borough     longitude       latitude  
0            EWR  9.366817e+05  190522.130278  
1         Queens  1.033536e+06  161853.982300  
2          Bronx  1.027136e+06  254730.010849  
3      Manhattan  9.904240e+05  203100.040432  
4  Staten Island  9.321332e+05  139954.541936

#### Remove location id's that are outside of the Manhattan

In [9]:
loc_df = loc_df[loc_df.borough == "Manhattan"]
loc_df.head()

OBJECTID  Shape_Leng  Shape_Area               zone  LocationID  \
3          4    0.043567    0.000112      Alphabet City           4   
11        12    0.036661    0.000042       Battery Park          12   
12        13    0.050281    0.000149  Battery Park City          13   
23        24    0.047000    0.000061       Bloomingdale          24   
40        41    0.052793    0.000143     Central Harlem          41   

      borough      longitude       latitude  
3   Manhattan  990424.019570  203100.040432  
11  Manhattan  979889.680922  195215.447490  
12  Manhattan  979844.241304  198508.159318  
23  Manhattan  993795.402611  231438.438453  
40  Manhattan  997895.956595  232264.009901

#### Convert location ids that belong to Manhattan into a list

In [11]:
manhattan_loc_id_list = loc_df["LocationID"].tolist()
print(manhattan_loc_id_list)

[4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 103, 103, 103, 103, 103, 103, 103, 103, 103, 107, 113, 114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]


## Intersect NYC Taxi and NYC Taxi Zone Data

#### Remove taxi rides that didn't originate from Manhattan

In [12]:
nyc_df = nyc_df[nyc_df['PULocationID'].isin(manhattan_loc_id_list)]

nyc_df.head()

VendorID tpep_dropoff_datetime  passenger_count  \
tpep_pickup_datetime                                                    
2018-01-01 00:21:05          1   2018-01-01 00:24:23                1   
2018-01-01 00:44:55          1   2018-01-01 01:03:05                1   
2018-01-01 00:08:26          1   2018-01-01 00:14:21                2   
2018-01-01 00:20:22          1   2018-01-01 00:52:51                1   
2018-01-01 00:09:18          1   2018-01-01 00:27:06                2   

                      trip_distance  RatecodeID store_and_fwd_flag  \
tpep_pickup_datetime                                                 
2018-01-01 00:21:05             0.5           1                  N   
2018-01-01 00:44:55             2.7           1                  N   
2018-01-01 00:08:26             0.8           1                  N   
2018-01-01 00:20:22            10.2           1                  N   
2018-01-01 00:09:18             2.5           1                  N   

                      PULocationID  DOLocationID  payment_type  fare_amount  \
tpep_pickup_datetime                                                          
2018-01-01 00:21:05             41            24             2          4.5   
2018-01-01 00:44:55            239           140             2         14.0   
2018-01-01 00:08:26            262           141             1          6.0   
2018-01-01 00:20:22            140           257             2         33.5   
2018-01-01 00:09:18            246           239             1         12.5   

                      extra  mta_tax  tip_amount  tolls_amount  \
tpep_pickup_datetime                                             
2018-01-01 00:21:05     0.5      0.5        0.00           0.0   
2018-01-01 00:44:55     0.5      0.5        0.00           0.0   
2018-01-01 00:08:26     0.5      0.5        1.00           0.0   
2018-01-01 00:20:22     0.5      0.5        0.00           0.0   
2018-01-01 00:09:18     0.5      0.5        2.75           0.0   

                      improvement_surcharge  total_amount  
tpep_pickup_datetime                                       
2018-01-01 00:21:05                     0.3          5.80  
2018-01-01 00:44:55                     0.3         15.30  
2018-01-01 00:08:26                     0.3          8.30  
2018-01-01 00:20:22                     0.3         34.80  
2018-01-01 00:09:18                     0.3         16.55

## Weather Data

#### Import the weather dataset

In [13]:
weather_df = pd.read_csv("datasets/weather_data.csv")
weather_df.head()

year  month  day  temp_high  temp_avg  temp_low  dew_high  dew_avg  \
0  2016      1    1          6         3         1      -3.0     -5.0   
1  2016      1    2          4         2         0      -7.0     -8.0   
2  2016      1    3          7         4         2      -5.0     -6.0   
3  2016      1    4          2        -4       -10      -5.0    -13.0   
4  2016      1    5         -2        -7       -12     -17.0    -19.0   

   dew_low  humidity_high  ...  sea_level_press_avg  sea_level_press_low  \
0     -9.0             59  ...               1017.0               1015.0   
1     -9.0             52  ...               1016.0               1014.0   
2     -7.0             56  ...               1011.0               1009.0   
3    -18.0             59  ...               1017.0               1011.0   
4    -21.0             58  ...               1035.0               1031.0   

   visibility_high  visibility_avg  visibility_low  wind_high  wind_avg  \
0             16.0            16.0            16.0       27.0      12.0   
1             16.0            16.0            16.0       24.0      11.0   
2             16.0            16.0            16.0       27.0      13.0   
3             16.0            16.0            16.0       34.0      14.0   
4             16.0            16.0            16.0       32.0       8.0   

   wind_high.1  precipitation  events  
0         42.0            0.0     NaN  
1         37.0            0.0     NaN  
2         42.0            0.0     NaN  
3         50.0            0.0     NaN  
4         48.0            0.0     NaN  

[5 rows x 23 columns]

#### Check the types of the columns

In [14]:
weather_df.dtypes

year                      int64
month                     int64
day                       int64
temp_high                 int64
temp_avg                  int64
temp_low                  int64
dew_high                float64
dew_avg                 float64
dew_low                 float64
humidity_high             int64
humidity_avg              int64
humidity_low              int64
sea_level_press_high    float64
sea_level_press_avg     float64
sea_level_press_low     float64
visibility_high         float64
visibility_avg          float64
visibility_low          float64
wind_high               float64
wind_avg                float64
wind_high.1             float64
precipitation           float64
events                   object
dtype: object

#### Convert events column into multiple columns

In [15]:
weather_df['event_rain'] = 0
weather_df['event_fog'] = 0
weather_df['event_snow'] = 0

for index, row in weather_df.iterrows():
    if(isinstance(weather_df.events[index], str)):
        if("Rain" in weather_df.events[index]):
            weather_df.event_rain[index] = 1

        if("Fog" in weather_df.events[index]):
            weather_df.event_fog[index] = 1

        if("Snow" in weather_df.events[index]):
            weather_df.event_snow[index] = 1


/Users/bsimsek/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/bsimsek/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/bsimsek/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Check for missing values

In [16]:
weather_df.isnull().sum()

year                      0
month                     0
day                       0
temp_high                 0
temp_avg                  0
temp_low                  0
dew_high                  2
dew_avg                   2
dew_low                   2
humidity_high             0
humidity_avg              0
humidity_low              0
sea_level_press_high      7
sea_level_press_avg       7
sea_level_press_low       7
visibility_high          10
visibility_avg           10
visibility_low           10
wind_high                88
wind_avg                 88
wind_high.1             100
precipitation            90
events                  670
event_rain                0
event_fog                 0
event_snow                0
dtype: int64

#### Drop rows with missing values and the events column since it is a string

In [17]:
weather_df = weather_df.dropna()

weather_df = weather_df.drop("events", axis = 1)

weather_df.isnull().sum()

year                    0
month                   0
day                     0
temp_high               0
temp_avg                0
temp_low                0
dew_high                0
dew_avg                 0
dew_low                 0
humidity_high           0
humidity_avg            0
humidity_low            0
sea_level_press_high    0
sea_level_press_avg     0
sea_level_press_low     0
visibility_high         0
visibility_avg          0
visibility_low          0
wind_high               0
wind_avg                0
wind_high.1             0
precipitation           0
event_rain              0
event_fog               0
event_snow              0
dtype: int64

#### Add primary key to be used in the NYC taxi data

In [18]:
weather_df.insert(0, "primary_key", "") 

for index, row in weather_df.iterrows():
    key = str(row['year'])
    
    if (row['month'] == 0) or (row['month'] == 1) or (row['month'] == 2) or (row['month'] == 3) or (row['month'] == 4) or (row['month'] == 5) or (row['month'] == 6) or (row['month'] == 7) or (row['month'] == 8) or (row['month'] == 9):
        key = key + "-0" + str(row['month'])
    else:
        key = key + "-" + str(row['month'])
        
    if (row['day'] == 0) or (row['day'] == 1) or (row['day'] == 2) or (row['day'] == 3) or (row['day'] == 4) or (row['day'] == 5) or (row['day'] == 6) or (row['day'] == 7) or (row['day'] == 8) or (row['day'] == 9):
        key = key + "-0" + str(row['day'])
    else:
        key = key + "-" + str(row['day'])
        
    weather_df.primary_key[index] = key

/Users/bsimsek/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [19]:
weather_df.head()

primary_key  year  month  day  temp_high  temp_avg  temp_low  dew_high  \
9   2016-01-10  2016      1   10         15        10         4      13.0   
15  2016-01-16  2016      1   16         11         8         6       5.0   
16  2016-01-17  2016      1   17          6         2        -1      -2.0   
21  2016-01-22  2016      1   22         -1        -3        -6      -8.0   
22  2016-01-23  2016      1   23         -3        -3        -4      -4.0   

    dew_avg  dew_low  ...  visibility_high  visibility_avg  visibility_low  \
9       8.0     -5.0  ...             16.0             7.0             1.0   
15      3.0     -2.0  ...             16.0            12.0             3.0   
16     -5.0     -8.0  ...             16.0            13.0             1.0   
21    -14.0    -16.0  ...             16.0            14.0             2.0   
22     -6.0     -9.0  ...              8.0             1.0             0.0   

    wind_high  wind_avg  wind_high.1  precipitation  event_rain  event_fog  \
9        35.0      16.0         56.0          45.72           1          0   
15       24.0      11.0         35.0           6.10           1          0   
16       21.0       9.0         31.0           1.27           0          1   
21       26.0      10.0         35.0           0.25           0          0   
22       40.0      25.0         68.0          58.67           0          1   

    event_snow  
9            0  
15           0  
16           1  
21           1  
22           1  

[5 rows x 26 columns]

## Add Weather Data to NYC Taxi Data

for index2, row in nyc_df.iterrows():
    if("2018-01-01" in nyc_df.index.day[index2]):
        print("ok")
    